# 在 Tool 和 Agent 之间共享记忆

- 自定义一个工具用来LLMChain来总结内容
- 使用readonlymemory来共享记忆
- 观察共享与不共享的区别

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper

1. 创建一条链来总结对话

In [ ]:
template = """以下是一段AI机器人和人类的对话:
{chat_history}
根据输入和上面的对话记录写一份对话总结.
输入: {input}"""

prompt = PromptTemplate(
    input_variables=["input","chat_history"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
readonlymemory = ReadOnlySharedMemory(memory=memory)

summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)

2. 构建工具

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#搜索工具
search = SerpAPIWrapper()
#总结工具
def SummaryChainFun(history):
    print("\n==============总结链开始运行==============")
    print("输入历史: ",history)
    summary_chain.run(history)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="当需要了解实时的信息或者你不知道的事时候可以使用搜索工具",
    ),
    Tool(
        name="Summary",
        func=SummaryChainFun,
        description="当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用",
    ),
]
print(tools)

3. 创建记忆工具

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

4. 创建 agent

In [ ]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

# 查看默认 prompt
print(agent_chain.agent.llm_chain.prompt.template)

# Answer the following questions as best you can. You have access to the following tools:

# Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
# Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [Search, Summary]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}
# Thought:{agent_scratchpad}

5. 重新定义 prompt，插入记忆

In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}"""

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix":prefix,
        "suffix":suffix,
        "agent_scratchpad":MessagesPlaceholder("agent_scratchpad"),
        "chat_history":MessagesPlaceholder("chat_history"),
        "input":MessagesPlaceholder("input"),
    },
    memory=memory,
)
print(agent_chain.agent.llm_chain.prompt.template)

# Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

# Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
# Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [Search, Summary]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# Begin!"
# {chat_history}
# Question: {input}
# {agent_scratchpad}

In [ ]:
agent_chain.run(input="美国第45任总统是谁?")
print(agent_chain.memory.buffer)

agent_chain.run(input="我们都聊了什么？")
print(agent_chain.memory.buffer)